<a href="https://colab.research.google.com/github/LuuZwide/Auto-Encoders/blob/main/Autoencoders_Testing_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import yfinance as yf
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/Functions/Preprocessing/Precossing_Functions_for_stock_chpters_1.ipynb" .
%run Precossing_Functions_for_stock_chpters_1.ipynb

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC, LinearSVR, SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve,roc_auc_score
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, MaxAbsScaler

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import tensorflow.keras.backend as K
from keras.activations import elu
from keras.engine.training import optimizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model

In [ ]:
ticker = yf.Ticker("AAPL")
apple_data = ticker.history(period="Max")
apple_data = apple_data.drop(['Dividends','Stock Splits'] , axis = 1)
apple_data.reset_index(drop = False, inplace = True)

ticker = yf.Ticker("IBM")
ibm_data = ticker.history(period="Max")
ibm_data = ibm_data.drop(['Dividends','Stock Splits'] , axis = 1)
ibm_data.reset_index(drop = False, inplace = True)

ticker = yf.Ticker("GE")
ge_data = ticker.history(period="Max")
ge_data = ge_data.drop(['Dividends','Stock Splits'] , axis = 1)
ge_data.reset_index(drop = False, inplace = True)

ticker = yf.Ticker("amzn")
amzn_data = ticker.history(period="Max")
amzn_data = amzn_data.drop(['Dividends','Stock Splits'] , axis = 1)
amzn_data.reset_index(drop = False, inplace = True)

ticker = yf.Ticker("GOOGL")
google_data = ticker.history(period="Max")
google_data = google_data.drop(['Dividends','Stock Splits'] , axis = 1)
google_data.reset_index(drop = False, inplace = True)

stock_pd = pd.concat([ibm_data,google_data,amzn_data,ge_data])
stock_pd = stock_pd.reset_index(drop = True)
stock_pd.head()
stock_pd.describe()

prices = stock_pd['Close']
len_of_data = len(prices)
window_size = 5
stock_dataset = Combine_dataframes(stock_pd ,prices, window_size,len_of_data, fast = 12, slow = 26)

y = stock_dataset['next_close']
X = stock_dataset.drop('next_close', axis = 1)
print(X)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)
attribs = list(x_train.columns)
#attribs = list(['Open', 'High', 'Low', 'Close', 'moving_avg', 'exm_avg'])
print(len(x_train))
print(len(x_test))

            Open        High        Low      Close   Volume  moving_avg  \
0       1.631550    1.631550   1.613218   1.613218   407940    0.000000   
1       1.613217    1.627319   1.613217   1.627319   305955    0.000000   
2       1.627319    1.627319   1.610398   1.611103   274575    0.000000   
3       1.608987    1.608987   1.576553   1.579374   384405    0.000000   
4       1.577964    1.577964   1.537070   1.549761   572685    1.596155   
...          ...         ...        ...        ...      ...         ...   
42087  95.620003   96.849998  95.480003  96.769997  6126100   94.904001   
42088  97.000000   98.519997  96.860001  98.470001  7110800   95.816000   
42089  98.580002   99.309998  98.419998  99.029999  5617000   96.802000   
42090  99.300003  100.470001  99.059998  99.760002  6912400   97.894000   
42091  99.980003  100.080002  99.129997  99.510002  4079400   98.708000   

         exm_avg  STOCHk_5_3_3  STOCHd_5_3_3  MACD_12_26_9  MACDh_12_26_9  \
0       1.613218      

In [ ]:
data_pipeline = Pipeline(
    (
        ('ConvertDataFrame',DataFrameSelector(attribs)),
        ('Scaler', MinMaxScaler())
    )
)

x_train_prepared = data_pipeline.fit_transform(x_train)
print('x_train shape: ',x_train_prepared.shape)
print('y_train shape: ',y_train.shape)

x_train shape:  (33673, 12)
y_train shape:  (33673,)


In [ ]:
lr = 0.001
encoding_dim = 5

input_layer = keras.layers.Input(shape = x_train_prepared.shape[1])
encoder_layer_one = keras.layers.Dense(32, activation = 'relu')(input_layer)
encoder_layer_two = keras.layers.Dense(16, activation = 'relu')(encoder_layer_one)
encoder = keras.layers.Dense(encoding_dim)(encoder_layer_two)

decoder_layer_one = keras.layers.Dense(16, activation = 'relu')(encoder)
decoder_layer_two = keras.layers.Dense(32, activation = 'relu')(decoder_layer_one)
decoder = keras.layers.Dense(x_train_prepared.shape[1])(decoder_layer_two)
autoencoder = Model(inputs = input_layer, outputs = decoder)

autoencoder.compile(optimizer = keras.optimizers.Adam(learning_rate = lr), loss= 'mean_squared_error')
autoencoder.fit(
    x = x_train_prepared,
    y = y_train,
    epochs = 50,
    batch_size = 128
)

Epoch 1/50
264/264 [==============================] - 2s 2ms/step - loss: 1081.4727
Epoch 2/50
264/264 [==============================] - 1s 5ms/step - loss: 3.0249
Epoch 3/50
264/264 [==============================] - 1s 5ms/step - loss: 2.4403
Epoch 4/50
264/264 [==============================] - 1s 3ms/step - loss: 2.3864
Epoch 5/50
264/264 [==============================] - 1s 3ms/step - loss: 2.3863
Epoch 6/50
264/264 [==============================] - 1s 3ms/step - loss: 2.3065
Epoch 7/50
264/264 [==============================] - 1s 2ms/step - loss: 2.3138
Epoch 8/50
264/264 [==============================] - 1s 2ms/step - loss: 2.2816
Epoch 9/50
264/264 [==============================] - 1s 2ms/step - loss: 2.2722
Epoch 10/50
264/264 [==============================] - 1s 2ms/step - loss: 2.3336
Epoch 11/50
264/264 [==============================] - 1s 2ms/step - loss: 2.2295
Epoch 12/50
264/264 [==============================] - 1s 2ms/step - loss: 2.2359
Epoch 13/50
264/264 [=